In [1]:
import pandas as pd
import yaml
from credit_default.utils import read_yaml_file
import os

In [4]:
df = pd.read_csv("/home/ajana/Code/MLProjects/credit_default/artifact/12_26_2023_01_56_22/data_ingestion/feature_store/UCI_Credit_Card.csv")
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [15]:
df.shape

(30000, 25)

['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_5', 'PAY_6', 'default.payment.next.month']


In [5]:
# Get column names and types as a dictionary
column_info = [{col: str(df[col].dtype)} for col in df.columns]

In [6]:
# Convert to a dictionary with a "columns" key
col_yaml_data = {'columns': column_info}
col_yaml_data

{'columns': [{'ID': 'int64'},
  {'LIMIT_BAL': 'float64'},
  {'SEX': 'int64'},
  {'EDUCATION': 'int64'},
  {'MARRIAGE': 'int64'},
  {'AGE': 'int64'},
  {'PAY_0': 'int64'},
  {'PAY_2': 'int64'},
  {'PAY_3': 'int64'},
  {'PAY_4': 'int64'},
  {'PAY_5': 'int64'},
  {'PAY_6': 'int64'},
  {'BILL_AMT1': 'float64'},
  {'BILL_AMT2': 'float64'},
  {'BILL_AMT3': 'float64'},
  {'BILL_AMT4': 'float64'},
  {'BILL_AMT5': 'float64'},
  {'BILL_AMT6': 'float64'},
  {'PAY_AMT1': 'float64'},
  {'PAY_AMT2': 'float64'},
  {'PAY_AMT3': 'float64'},
  {'PAY_AMT4': 'float64'},
  {'PAY_AMT5': 'float64'},
  {'PAY_AMT6': 'float64'},
  {'default.payment.next.month': 'int64'}]}

In [29]:
cat_columns = []
threshold = 12  # Set the threshold for the number of unique values

for column in df.columns:
    unique_values = df[column].nunique()
    print (column, unique_values)
    if unique_values <= threshold:
        cat_columns.append(column)

print(cat_columns)

ID 30000
LIMIT_BAL 81
SEX 2
EDUCATION 7
MARRIAGE 4
AGE 56
PAY_0 11
PAY_2 11
PAY_3 11
PAY_4 11
PAY_5 10
PAY_6 10
BILL_AMT1 22723
BILL_AMT2 22346
BILL_AMT3 22026
BILL_AMT4 21548
BILL_AMT5 21010
BILL_AMT6 20604
PAY_AMT1 7943
PAY_AMT2 7899
PAY_AMT3 7518
PAY_AMT4 6937
PAY_AMT5 6897
PAY_AMT6 6939
default.payment.next.month 2
['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'default.payment.next.month']


In [22]:
# Filter numerical columns
numerical_columns = df.select_dtypes(include=['number']).columns.to_list()
numerical_columns = [col for col in numerical_columns if col not in cat_columns]
# Convert to a dictionary with a "numerical_columns" key
num_yaml_data = {'numerical_columns': numerical_columns}


In [23]:
target = {'target_column': 'default.payment.next.month'}


In [24]:
col_yaml_data

{'columns': [{'ID': 'int64'},
  {'LIMIT_BAL': 'float64'},
  {'SEX': 'int64'},
  {'EDUCATION': 'int64'},
  {'MARRIAGE': 'int64'},
  {'AGE': 'int64'},
  {'PAY_0': 'int64'},
  {'PAY_2': 'int64'},
  {'PAY_3': 'int64'},
  {'PAY_4': 'int64'},
  {'PAY_5': 'int64'},
  {'PAY_6': 'int64'},
  {'BILL_AMT1': 'float64'},
  {'BILL_AMT2': 'float64'},
  {'BILL_AMT3': 'float64'},
  {'BILL_AMT4': 'float64'},
  {'BILL_AMT5': 'float64'},
  {'BILL_AMT6': 'float64'},
  {'PAY_AMT1': 'float64'},
  {'PAY_AMT2': 'float64'},
  {'PAY_AMT3': 'float64'},
  {'PAY_AMT4': 'float64'},
  {'PAY_AMT5': 'float64'},
  {'PAY_AMT6': 'float64'},
  {'default.payment.next.month': 'int64'}]}

In [25]:
yaml_data = {}
yaml_data['columns'] = column_info
yaml_data['numerical_columns'] = numerical_columns
yaml_data['drop_columns'] = 'ID'
yaml_data['target_column'] = 'default.payment.next.month'

In [26]:
# Save to YAML file
with open('/home/ajana/Code/MLProjects/credit_default/config/schema.yaml', 'w') as file:
    yaml.dump(yaml_data, file, default_flow_style=False, sort_keys=False)

In [27]:
SCHEMA_FILE_PATH = "/home/ajana/Code/MLProjects/credit_default/config/schema.yaml"

schema_config = read_yaml_file(file_path=SCHEMA_FILE_PATH)


In [28]:
schema_config["target_column"]

'default.payment.next.month'